In [ ]:
#import packages
import scipy.linalg as scilin
import numpy as np
import json


In [ ]:
#Functions Section

In [8]:
#Read in the data from the JSON file
filename_load = "0000.json"
J_string = open(filename_load).read()
J_vals = json.loads(J_string)

#################  Aircraft Properties  ##################################

#Get the moment information of aircraft
I_xx = J_vals['aircraft']['Ixx[slugs*ft^2]']
I_yy = J_vals['aircraft']['Iyy[slugs*ft^2]']
I_zz = J_vals['aircraft']['Izz[slugs*ft^2]']
I_xy = J_vals['aircraft']['Ixy[slugs*ft^2]']
I_xz = J_vals['aircraft']['Ixz[slugs*ft^2]']

#Name
Plane_name = J_vals['aircraft']['name']

#Wing Information
Wing_Area = J_vals['aircraft']['wing_area[ft^2]']
Wing_Span = J_vals['aircraft']['wing_span[ft]']

#aircraft weight
Weight = J_vals['aircraft']['weight[lbf]']

#Launch Energy
Launch_Energy = J_vals['aircraft']['launch_kinetic_energy[ft-lbf]']

#################  Air Properties  ########################################

#density
Air_Density = J_vals['analysis']['density[slugs/ft^3]']

#################  Aerodynamic Properties  #################################

#CL values
CL_0 = J_vals['aerodynamics']['CL']['0']
CL_alpha = J_vals['aerodynamics']['CL']['alpha']
CL_qbar = J_vals['aerodynamics']['CL']['qbar']
CL_alpha_hat = J_vals['aerodynamics']['CL']['alpha_hat']

#CY values
CY_beta = J_vals['aerodynamics']['CY']['beta']
CY_pbar = J_vals['aerodynamics']['CY']['pbar']
CY_rbar = J_vals['aerodynamics']['CY']['rbar']

#CD values
CD_L0 = J_vals['aerodynamics']['CD']['L0']
CD_L1 = J_vals['aerodynamics']['CD']['L']
CD_L2 = J_vals['aerodynamics']['CD']['L2']
CD_0 = CD_L0 + CD_L1*CL_0*CD_L2*CL_0**2
CD_alpha = CD_L1*CL_alpha + 2*CD_L2*CL_0*CL_alpha
CD_qbar = J_vals['aerodynamics']['CD']['qbar']

#Cl Values
Cl_beta = J_vals['aerodynamics']['Cl']['beta']
Cl_pbar = J_vals['aerodynamics']['Cl']['pbar']
Cl_rbar = J_vals['aerodynamics']['Cl']['rbar']

#Cm Values
Cm_0 = J_vals['aerodynamics']['Cm']['0']
Cm_alpha = J_vals['aerodynamics']['Cm']['alpha']
Cm_qbar = J_vals['aerodynamics']['Cm']['qbar']
Cm_alpha_hat = J_vals['aerodynamics']['Cm']['alpha_hat']

#Cn values
Cn_beta = J_vals['aerodynamics']['Cn']['beta']
Cn_pbar = J_vals['aerodynamics']['Cn']['pbar']
Cn_rbar = J_vals['aerodynamics']['Cn']['rbar']



0.0708
{'aircraft': {'name': 'Baseline Glider', 'launch_kinetic_energy[ft-lbf]': 4.5, 'wing_area[ft^2]': 6.75, 'wing_span[ft]': 9, 'weight[lbf]': 1.1013, 'Ixx[slugs*ft^2]': 0.0708, 'Iyy[slugs*ft^2]': 0.0527, 'Izz[slugs*ft^2]': 0.1196, 'Ixy[slugs*ft^2]': 0.0, 'Ixz[slugs*ft^2]': 0.0023, 'Iyz[slugs*ft^2]': 0.0}, 'analysis': {'density[slugs/ft^3]': 0.0020664}, 'aerodynamics': {'CL': {'0': 0.628, 'alpha': 5.478, 'qbar': 2.3594, 'alpha_hat': 0.7557}, 'CY': {'beta': -0.3795, 'pbar': -0.3336, 'rbar': 0.3481}, 'CD': {'L0': 0.0138, 'L': -0.00753, 'L2': 0.0391, 'qbar': 0.1241}, 'Cl': {'beta': -0.1423, 'pbar': -0.6309, 'rbar': 0.2747}, 'Cm': {'0': -0.0046, 'alpha': -1.3102, 'qbar': -9.7591, 'alpha_hat': -1.9067}, 'Cn': {'beta': 0.0856, 'pbar': -0.053, 'rbar': -0.0793}}}
0.013727079327568704
0.22777304880000002


In [ ]:
#Generating the Matrices